# <center> Reading, math and writing demonstration

### <center> In this demo you will use the tProc to do some math and/or bitwise operations on two variables that you will pass in through the <code> config </code> dictionary and take out with <code> single_read </code>.

In [1]:
# Import the QICK drivers and auxiliary libraries
from qick import *
from qick.qick_asm import *
from qick.averager_program import *
from qick.helpers import gauss
import time
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Load bitstream with custom overlay
soc = QickSoc()
# Set the loopback DAC channel to be in 1st Nyquist zone mode
soc.set_nyquist(ch=7,nqz=1); 

### Hardware Configuration

tProc channel 7   : DAC 229 CH3  <-> Readout channel 0 : ADC 224 CH0

In [3]:
class LoopbackProgram(AveragerProgram):
    def __init__(self,cfg):
        AveragerProgram.__init__(self,cfg)

    def initialize(self):
        cfg=self.cfg   

        # Move value of A into tProc register
        self.regwi(0,1,self.cfg["A"])

        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        # Do math: add value of B to the register value
        self.mathi(0, 1, 1, '+', self.cfg["B"])
        
        # Write the result to address 123
        self.memwi(0,1,123)

### Here we will add the value of  <code> A </code> to the value of <code> B </code> using the tProc. We write the result to tProc memory address 123, and then read out that address value at the end of the program.

In [4]:
config={"reps": 1, # fixed
        "A": 100,
        "B": 50
       }

prog =LoopbackProgram(config)

soc.tproc.load_qick_program(prog)
# Start tProc.
soc.tproc.stop()
soc.tproc.start()
time.sleep(0.1)
result = soc.tproc.single_read(addr=123)
print("Result = ", result)

Result =  150
